- **Name:** 001_Introduction
- **Author:** Shamas Imran
- **Desciption:** Implementing aggregations using dataframe

In [0]:
from pyspark.sql.types import *
from pyspark.sql import Row
from datetime import date 
import random
from pyspark.sql import functions as F
from pyspark.sql.window import Window

# -------------------------------
# 1. Student DataFrame
# -------------------------------
student_schema = StructType([
    StructField('StudentID', IntegerType(), False),
    StructField('StudentName', StringType(), True),
    StructField('StudentAge', IntegerType(), True)
])

student_data = [
    (1, "Alice", 34), 
    (2, "Bob", 45), 
    (3, "Charlie", 29),
    (4, "Shamas", 40)
]

df_student = spark.createDataFrame(student_data, student_schema)

# -------------------------------
# 2. Course DataFrame
# -------------------------------
course_schema = StructType([
    StructField('CourseID', IntegerType(), False),
    StructField('CourseName', StringType(), True),
    StructField('CourseTitle', StringType(), True),
])

course_data = [
    (1, "Physics", "1111"), 
    (2, "Chemistry", "2222"), 
    (3, "English", "3333"),
    (4, "Computer Science", "4444")
]

df_course = spark.createDataFrame(course_data, course_schema)

# -------------------------------
# 3. Enrollment DataFrame
# -------------------------------
enrollment_schema = StructType([
    StructField("EnrollmentID", IntegerType(), False),
    StructField("StudentID_FK", IntegerType(), False),
    StructField("CourseID_FK", IntegerType(), False),
    StructField("EnrollmentDate", DateType(), True)
])

enrollment_data = [
    (1, 1, 1, date(2023, 9, 1)),   # Alice -> Physics
    (2, 2, 2, date(2023, 9, 2)),   # Bob -> Chemistry
    (3, 4, 4, date(2023, 9, 4)),   # Shamas -> Computer Science
    (4, 1, 2, date(2023, 9, 5)),   # Alice -> Chemistry
]

df_enrollment = spark.createDataFrame(enrollment_data, enrollment_schema)

# -------------------------------
# 4. Score DataFrame
# -------------------------------
semesters = ["2023-Spring", "2023-Fall", "2024-Spring", "2024-Fall", "2025-Spring"]
score_data = []
score_id = 1
enrollment_ids = [row.EnrollmentID for row in df_enrollment.collect()]

for enrollment_id in enrollment_ids:
    selected_semesters = random.sample(semesters, k=random.randint(2, 4))
    for sem in selected_semesters:
        score_data.append(Row(
            ScoreID=score_id,
            EnrollmentID_FK=enrollment_id,
            Semester=sem,
            Score=random.randint(60, 100)
        ))
        score_id += 1

score_schema = StructType([
    StructField("ScoreID", IntegerType(), False),
    StructField("EnrollmentID_FK", IntegerType(), False),
    StructField("Semester", StringType(), True),
    StructField("Score", IntegerType(), True)
])

df_score = spark.createDataFrame(score_data, schema=score_schema)

In [0]:
# Define row window: partition by course, order by score
row_window = (
    Window.partitionBy("e.CourseID_FK")
    .orderBy("s.Score")
    .rowsBetween(-2, 2)  # 2 rows before, current row, 2 rows after
)

df_score.alias("s") \
    .join(df_enrollment.alias("e"), F.col("s.EnrollmentID_FK") == F.col("e.EnrollmentID")) \
    .withColumn("MovingAvg", F.avg("s.Score").over(row_window)) \
    .select("e.CourseID_FK", "s.Score", "MovingAvg") \
    .show()

In [0]:
# Example 1: Running average score per course
df_score.alias("s") \
    .join(df_enrollment.alias("e"), F.col("s.EnrollmentID_FK") == F.col("e.EnrollmentID")) \
    .withColumn("RunningAvg", F.avg("s.Score").over(course_window)) \
    .select("e.CourseID_FK", "s.Score", "RunningAvg") \
    .show()

In [0]:
# Example 2: Running Total (from first row to current row)
running_window = (
    Window.partitionBy("e.CourseID_FK")
    .orderBy("s.Score")
    .rowsBetween(Window.unboundedPreceding, 0)  # everything up to current row
)

df_score.alias("s") \
    .join(df_enrollment.alias("e"), F.col("s.EnrollmentID_FK") == F.col("e.EnrollmentID")) \
    .withColumn("RunningTotal", F.sum("s.Score").over(running_window)) \
    .select("e.CourseID_FK", "s.Score", "RunningTotal") \
    .show()

In [0]:
forward_window = (
    Window.partitionBy("e.CourseID_FK")
    .orderBy("s.Score")
    .rowsBetween(0, Window.unboundedFollowing)  # current row to end
)

df_score.alias("s") \
    .join(df_enrollment.alias("e"), F.col("s.EnrollmentID_FK") == F.col("e.EnrollmentID")) \
    .withColumn("ForwardTotal", F.sum("s.Score").over(forward_window)) \
    .select("e.CourseID_FK", "s.Score", "ForwardTotal") \
    .show()

In [0]:
# Example 4: Row Difference (current row vs previous row)
diff_window = (
    Window.partitionBy("e.CourseID_FK")
    .orderBy("s.Score")
    .rowsBetween(-1, 0)  # previous row + current row
)

df_score.alias("s") \
    .join(df_enrollment.alias("e"), F.col("s.EnrollmentID_FK") == F.col("e.EnrollmentID")) \
    .withColumn("RowDiff", (F.sum("s.Score").over(diff_window) - F.col("s.Score"))) \
    .select("e.CourseID_FK", "s.Score", "RowDiff") \
    .show()